# Writing Functions in Python

## Learning points:

* Documentation
    * Docstrings
* Principles 
    * DRY (Don't Repeat Yourself)
        * E.g.: Decorators (add common behavior to multiple functions instead of replicating it in the body of each function)
    * Do One Thing
    * Pass by Assignment
        * Example: Lists (Mutable) vs Integers (Immutable)
        * Example: Mutable Default Argument Newbie Mistake
* Context Manager
    * It can be defined in 2 ways
        * Function (a function-based Context Manager)
        * Class (a class-based Context Manager)
* Decorators
    * Functions as objects
        * Functions as variables
        * Function as items of a list or a dictionary
        * Function as arguments of another function
        * Nested Functions
        * Function as a returned value of a function
    * Scope
        * Which variables can be accessed at different points in the code
            * global, local, nonlocal
        * Variable names inside and outside the function
            * local scope first, nonlocal-but-nested next, global next, builtin next (e.g.: print() function)
    * Closures
        * A tuple of variables that are no longer in scope (but a function still needs it to run its code)
            * Even if you delete a global variable, the previous value that was stored in the function's closure remains in the tuple that is created behind the scenes
    * Summary of Decorators
        * A decorator is a function (e.g., my_decorator_function()) that wraps another function with additional behavior.
        * It contains a nested function (often called wrapper) that modifies or enhances the behavior of the decorated function.
        * You apply a decorator to another function using the @ symbol (e.g., @my_decorator_function), which automatically replaces the original function with the decorated version.
        * This process is similar to composite functions in mathematics, where the decorator acts as an outer function that transforms or modifies the inner function’s behavior.
        * If you need to access the original, undecorated function, you can use the __wrapped__ attribute (e.g., result = my_function.__wrapped__(*args, **kwargs)).
            * But this only works if `functools.wraps` was used in the decorator.  

## Documentation (Docstring)

```
def function_name(arguments):
    """
    Description of what the function does.
    Description of the arguments, if any.
    Description of the return value(s), if any.
    Description of errors raised, if any.
    Optional extra notes or examples of usage.
    """
```

In [1]:
def my_function(arg_1, arg_2=10):
    """
    Description of what the function does.

    Args:    
        arg_1 (str): Description of arg_1 that can break onto the next line
        if needed.
        arg_2 (int, optional): Write optional when an argument has a default
        value.  

    Returns:    
        int: Optional description of the return value
        Extra lines are not indented.

    Raises:
        ValueError: Include any error types that the function intentionally
        raises.
        
    Notes:
        See XYZ document for more info.
    """

# To access the docstring, use the __doc__ attribute of the function
print(my_function.__doc__)




    Description of what the function does.

    Args:    
        arg_1 (str): Description of arg_1 that can break onto the next line
        if needed.
        arg_2 (int, optional): Write optional when an argument has a default
        value.  

    Returns:    
        int: Optional description of the return value
        Extra lines are not indented.

    Raises:
        ValueError: Include any error types that the function intentionally
        raises.
        
    Notes:
        See XYZ document for more info.
    


In [2]:
# We can also use the inspect module to get the docstring
import inspect
print(inspect.getdoc(my_function))

Description of what the function does.

Args:    
    arg_1 (str): Description of arg_1 that can break onto the next line
    if needed.
    arg_2 (int, optional): Write optional when an argument has a default
    value.  

Returns:    
    int: Optional description of the return value
    Extra lines are not indented.

Raises:
    ValueError: Include any error types that the function intentionally
    raises.
    
Notes:
    See XYZ document for more info.


## Principles

### Pass by Assignment and The "Mutable Default Argument" Newbie Mistake

In Python, everything is an `object`, and variables are just references (`pointers`) to these objects.

When you pass a variable to a function, Python assigns the reference to the parameter — this is called `Pass by Assignment`.

* Difference Between `Objects` and `Variables` in Python
    * `Objects`   → The actual **data stored in memory**.
        * Each object has a memory address (location)
        * Objects are created when assigned or instantiated (x = 10, my_list = [])
    * `Variables` → References (**pointers**) to objects in memory.
        * A variable is just a name that points to an object in memory.
        * **Variables don’t store the object** itself, **they store a reference** (memory address) to the object

It is easier to understand if we focus on `Immutable vs. Mutable` objects.

#### Example 1: Immutable Object (No Change Outside the Function)

* `int` is immutable, so `x = x + 1` creates a new object, leaving `num` unchanged

In [3]:
def modify_number(x):
    x = x + 1  # Creates a NEW object
    print("Inside function:", x)

num = 10
modify_number(num)
print("Outside function:", num)  # Still 10 (unchanged)

Inside function: 11
Outside function: 10


Here, what happened is the following:
* `num = 10`
    * `num` points to an `integer` object `10` in memory, let's say memory address in this case is `A100`
* `modify_number(num)`
    * The function receives **a copy** of the reference (**not the value**)
    * `x` is created inside the function and x also points to `10` at `A100`
* So far, both `num` and `x` point to `10` at `A100`
* `x = x + 1`
    * now, a new object is created. It is a new integer `11` at a different memory address (`A200`)
* So far, `x` now points to `11` (`A200`), but `num` still points to `10` (`A100`)
* After function execution is done:
    * x (inside the function) disappears
    * `num` is unchanged, still pointing to `10` (`A100`)

Flow:
```
num --> [10] (A100)   # object 10 is created and num points to it at memory A100
x   --> [10] (A100)   # x is created inside the function and initially points to the same integer 10 at A100
x   --> [11] (A200)   # New integer 11 is created at a new location A200, x now points to it
num --> [10] (A100)   # num is unchanged, still pointing to A100
x   (deleted)         # x goes out of scope (no longer exists)
```

#### Example 2: Mutable Object  (Changes Affect the Original)

* `list` is mutable, so changes inside the function affect the original object

In [4]:
def modify_list(lst):
    lst.append(4)  # Modifies the existing object
    print("Inside function:", lst)

my_list = [1, 2, 3]
modify_list(my_list)
print("Outside function:", my_list)  # Now [1, 2, 3, 4] (changed!)

Inside function: [1, 2, 3, 4]
Outside function: [1, 2, 3, 4]


Here, what happened is the following:
* `my_list = [1, 2, 3]`
    * Python creates a `list` object `[1, 2, 3]` in memory at `A300`
* `modify_list(my_list)`
    * The function receives the `reference` to the same list (`A300`)
    * `lst` and `my_list` both point to the same memory address (`A300`)
* So far, both `lst` and `my_list` point to `[1, 2, 3]` at `A300`
* `lst.append(4)`
    * now, a new object is **NOT** created. Modifies the existing list at `A300`
* So far, both `lst` and `my_list` **STILL** point to `A300` but now the list is `[1, 2, 3, 4]` 
* After function execution is done:
    * lst **DOES NOT** disappear
    * `my_list` still points to `A300`, but now it contains `[1, 2, 3, 4]`

Flow:
```
my_list              --> [1, 2, 3]    (A300)  # List [1,2,3] is created at A300 and my_list points to it
modify_list(my_list) --> [1, 2, 3]    (A300)  # Function receives reference A300 to list [1, 2, 3] from the argument mylist
mylist=lst           --> [1, 2, 3]    (A300)  # lst is created and points to the same object (mylist) A300
lst.append(4)        --> [1, 2, 3, 4] (A300)  # The mutable object is modified and lst still points to A300
my_list              --> [1, 2, 3, 4] (A300)  # mylist still points to A300, which was modified via lst, so mylist was modified
```

#### Example - The Mutable Default Argument Newbie Mistake - Mutable Object as Argument

* If you want a Mutable object as a default argument, you need to Default it to None and initiate it inside the function.

In [5]:
def mistake(var=[]):  
    var.append(1)
    return var

# Call the function
print(mistake())
# Call it again
print(mistake())

# The default value is shared between calls
def no_mistake(var=None):
    if var is None:
        var = []  
    var.append(1)
    return var

print(no_mistake())
print(no_mistake())


[1]
[1, 1]
[1]
[1]


In [6]:
# Example: Function that takes a mutable object (list of values) and adds it to dataframe as a new column

import pandas as pd
data = [1, 2, 3, 4, 5]

def add_column(values, df: None):
    if df is None:
        df = pd.DataFrame()
    df['col_{}'.format(len(df.columns))] = values
    return df

add_column(data, df=None)

,col_0
0,1
1,2
2,3
3,4
4,5


## Context Manager = Special Type of Function

* What is a Context Manager?
    * It is a special type of **function** that:
        1) Sets up a context for your code to run in
        2) Runs the code
        3) Removes the context
    * The `with` statement is a **Compound Statement**, just like `if` or `for` loops (it runs the **INDENTED** code below)
    * How to return a value?
        * You can assign the returned value by using the variable name in the `as <variable_name>` part

* There are 2 types of use for Context Managers, i.e., Context Managers can be defined in 2 ways
    * Function (a function-based Context Manager)
        * In this case, tecnically, a `generator` that yields a single value
        * This ability of "yield" control and know that it will finish later is why Context Managers are useful
    * Class (a class-based Context Manager)

### Context Manager as a Function

Mathematically, it would be something like:
```
with f(x) as y:
 ...
 ```
Above, f(x) is the context manager function/object, and y is the return value.

In python:
```
with <context-manager, aka any function!>(<args>) as <variable-name>: #
  # Code inside the context

# This code runs after the context is removed
```

In [7]:
import contextlib

In [8]:
with open("C:/Users/caiov/OneDrive - UCLA IT Services/Documentos/DataScience/Repositories/writing-python-functions-best-practices/writing_python_functions_project/my_text.txt") as my_text: # Sets up a context manager (as a Compound Statement) by opening the file and assigns its value in the my_file variable
    text = my_text.read()
    length = len(text)

print('The file has this text:\n', text)
print('The file is {} characters long'.format(length))

The file has this text:
 It always seems impossible until it's done. - Nelson Mandela
The file is 60 characters long


### Writing Context Managers for other people to use
* First, you use the `decorator`: @contextlib.contextmanager
* Then, you define the function.
    * yield <value>: you will return a value, but at some point in the future
    * This value can be assined to variable in the `with` compound statement by adding `as <variable-name>`

```
@contextlib.contextmanager
def my_context():
    # Add any set up code you need
    yield <value>
    # Add any teardown code you need

with my_context_manager() as my_variable:
    print('the variable from yield is {}'.format(my_variable))
```     

In [9]:
# First Example
@contextlib.contextmanager  # This decorator allows defining a context manager using a generator function instead of a class
def my_context_manager():
                            # The yield statement defines the __enter__ and __exit__ methods
                            # The function pauses at yield and returns 10 to my_variable
    yield 10                
                            # After the with block below finishes printing, the function resumes after yield and prints the teardown code
    print('teardown code')

with my_context_manager() as my_variable:   # my_variable receives the value 10 from yield
    print('the variable from yield is {}'.format(my_variable))

the variable from yield is 10
teardown code


In [10]:
# Second Example - Yield 'None' to the variable
import os

@contextlib.contextmanager
def in_dir(path):
    # save current working directory  
    old_dir = os.getcwd()
    print("This is the old/current dir: ", old_dir)
    # switch to new working directory  
    os.chdir(path)
    print("This is the new dir, from with argument: ", path)
    yield
    # change back to previous working directory  
    os.chdir(old_dir)

with in_dir("C:/Users/caiov/OneDrive - UCLA IT Services/Documentos/DataScience/Repositories/writing-python-functions-best-practices/writing_python_functions_project/"):  
    project_files = os.listdir()
    print(project_files)


This is the old/current dir:  c:\Users\caiov\OneDrive - UCLA IT Services\Documentos\DataScience\Repositories\writing-python-functions-best-practices\writing_python_functions_project
This is the new dir, from with argument:  C:/Users/caiov/OneDrive - UCLA IT Services/Documentos/DataScience/Repositories/writing-python-functions-best-practices/writing_python_functions_project/
['my_text.txt', 'writing_python_functions_project.ipynb']


In [11]:
# Third Example - Accessing a Database

# @contextlib.contextmanager

# def database(url): 
#     # set up database connection
#     db = postgres.connect(url)
#     yield db
#     # tear down database connection 
#     db.disconnect()url = 'http://datacamp.com/data'
    
# with database(url) as my_db:
#     course_list = my_db.execute(
#         'SELECT * FROM courses'
#     )

### Nested Context Managers, Handling Error (Try-Except-Finally)

* Example of 2 nested context managers
    1) Context Manager for stock price connection and to get the text from the connection
    2) Context Manager to write in the opened the content from the connection to new text file

In [12]:
# Use the "stock('NVDA')" context manager
# and assign the result to the variable "nvda"
# with stock('NVDA') as nvda:
#   # Open "NVDA.txt" for writing as f_out
#   with open('NVDA.txt', 'w') as f_out:
#     for _ in range(10):
#       value = nvda.price()
#       print('Logging ${:.2f} for NVDA'.format(value))
#       f_out.write('{:.2f}\n'.format(value))

## Decorators

* Functions as objects
    * Functions as variables
    * Function as items of a list or a dictionary
    * Function as arguments of another function
    * Nested Functions
    * Function as a returned value of a function
* Scope
    * Which variables can be accessed at different points in the code
    * Variable names inside and outside the function
* Closures

### Functions as objects

* Function as variables
    * In this case, you **do not** include parenthesis after the function name
        * If you call with parenthesis, it shows the value that the function returns
        * If you call without parenthesis, it shows a **function object**

In [13]:
def my_function():
    return 10

x = my_function()

print(my_function())  # Here we are calling the function by using the parenthesis. It will return the value.
print(x)

print(my_function) # Here we are just printing the function object. It will return the memory address of the function object.



10
10
<function my_function at 0x000001DB32139260>


* Function as items of a list or a dictionary

In [14]:
def my_function():
    return 10

list_of_functions = [my_function, print]
print(list_of_functions[0]) # Returns the my_function object
print(list_of_functions[1]) # Returns the print function object
list_of_functions[1]("Hello") # Calls the print function with the argument "Hello"

dict_of_functions = {'f_1': my_function,'f2': print}
dict_of_functions['f2']('Hello') # Calls the print function with the argument "Hello"


<function my_function at 0x000001DB32139300>
<built-in function print>
Hello
Hello


* Function as arguments of another function
* Nested Functions = Inner Functions = Child Functions

In [15]:
# Function as Arguments
def my_function():
    return 10

def call_function(func):
    return func() # Calls the function that is passed as an argument (not the parenthesis here)

call_function(my_function) # Calls the my_function function

10

* Function as Return Values (aka: Returning a Function via another's function return value)

In [16]:
# Function as Return Values

def get_function():
    def print_message():
        print("Hello from print_message inside get_function")
    return print_message

new_function = get_function()
new_function() # Calls the print_message function

Hello from print_message inside get_function


In [17]:
import pandas as pd
import numpy as np

# Define function mappings to Pandas' built-in DataFrame methods
function_map = {
    'mean': lambda df: df.mean(),  # Compute column-wise means
    'std': lambda df: df.std(),
    'minimum': lambda df: df.min(),
    'maximum': lambda df: df.max()
}

data = {'a': [1, 2, 3, 4, 5], 'b': [5, 4, 3, 2, 1]}
df = pd.DataFrame(data)

# Suppose the user chooses the 'mean' by typing its name in a prompt
user_input = 'mean'  # This should be a string

# Call the chosen function and pass "df" as an argument
result = function_map[user_input](df)

print(type(result))
print(result)

<class 'pandas.core.series.Series'>
a    3.0
b    3.0
dtype: float64


### Scope
    
* Which variables can be accessed at different points in the code
* Variable names inside and outside the function
    * local scope first, nonlocal-but-nested next, global next, builtin next (e.g.: print() function)

In [18]:
# Working with scopes

x = 99     # the global scope is set to 99

def a():
  x = 20   # the local scope is set to 20

def b():
  global x # the global scope is now bring called
  x = 30   # the global scope is set to 30

def c():
  x = 50  # the local scope is set to 50
  print(x) # prints 50

for func in [a, b, c]:
  func() # calls each function
  print(x) # prints the global scope value after each function call

99
30
50
30


* Closure
    * It is python's way to attach a nonlocal variable to a returned function even when it is called outside of its parent's scope
    * A tuple of variables that are no longer in scope (but a function still needs it to run its code)
    * Even if you delete a global variable, the previous value that was stored in the function's closure remains in the tuple that is created behind the scenes

In [19]:
# This function gets the value of the global variable "x" and prints it via the child function
# This is where we understand the concept of closures

x = 10 # Global variable

def parent_function(value):  
    def child_function():
        print(value)
    return child_function

function_as_object = parent_function(x) # Calls the parent_function and assigns the return value to the variable
print(type(function_as_object))         # Returns the type of the object
print(function_as_object)               # Returns the object itself
function_as_object()                    # Calls the parent_function and the child_function inside it and prints the value of the global variable "x"

print(type(function_as_object.__closure__)) # Returns the type of the closure attribute of the function_as_object

del(x) # Deletes the global variable "x"
function_as_object() # It still prints the value of the global variable "x" because the child_function has access to the parent_function scope

print(len(function_as_object.__closure__)) # Returns the number of variables in the closure attribute
function_as_object.__closure__[0].cell_contents # Returns the value of the variable in the closure attribute

<class 'function'>
<function parent_function.<locals>.child_function at 0x000001DB321393A0>
10
<class 'tuple'>
10
1


10

In [20]:
def parent_function(arg_1, arg_2):  
  value = 22  
  my_dict = {'chocolate': 'yummy'}
  def child():
      print(2 * value)
      print(my_dict['chocolate'])
      print(arg_1 + arg_2)
  return child
    
new_function = parent_function(3, 4)

print([cell.cell_contents for cell in new_function.__closure__])

[3, 4, {'chocolate': 'yummy'}, 22]


### Finally: Decorators

* Decorators are just functions that take a function as argument and return a modified version of that function.
* Therefore, you are "decorating" a function
    * The code of the "decorator" function, comes as a "nested" function called "wrapper" (this name is used as best practices)

Goal: To add common behavior to multiple function instead of having to write again in the body of each new function.

It uses all the below:
* Function as objects
* Nested functions
* Nonlocal scope
* Closures

In [21]:
# One function will decorate the other function
# In this case, the decorator will not change the behavior of the multiply function
def multiply(a, b):
    return a * b

def double_args(func):
    return func

new_multiply = double_args(multiply)

new_multiply(1, 5)

5

In [22]:
# One function will decorate the other function
# In this case, the decorator will change the behavior of the multiply function
def multiply(a, b):
    return a * b

def double_args(func):  # This is a decorator function to modify the behavior of another function
    # Define a new function that we can modify (this is best practices)
    def wrapper(a, b):  # All this function does is call the original function with double the arguments
        # For now, just call the unmodified function
        return func(a * 2, b * 2)
    # Return the new function via the decorator
    return wrapper

new_multiply = double_args(multiply) # we are "decorating" the multiply function
new_multiply(1, 5) # This will return 20


20

In [23]:
# We can do the above in a more concise way using the decorator syntax
def double_args(func):
    def wrapper(a, b):
        return func(a * 2, b * 2)
    return wrapper

@double_args        # This is the same as saying multiply = double_args(multiply)
def multiply(a, b):
    return a * b

multiply(1, 5) # This will return 20

20

* Note for the example below:
    * `*args`
        * `*args` allows a function to accept a variable number of positional arguments
        * It collects all extra positional arguments passed to the function into a tuple.

    * `**kwargs`
        * **kwargs allows a function to accept a variable number of keyword arguments.
        * It collects all extra keyword arguments (those passed using the key=value syntax) into a dictionary.
        * Ex: foo(value=42)   
            * This would be a keyword argument. Here, kwargs would be {'value': 42}.
    * Creating attributes to a python function

In [24]:
# Quick example of *args

def sum_numbers(*args):
    return sum(args)

# Calling the function with different numbers of arguments
print(sum_numbers(1, 2, 3))          # Output: 6
print(sum_numbers(10, 20, 30, 40))   # Output: 100
print(sum_numbers(5))                # Output: 5
print(sum_numbers())                 # Output: 0

6
100
5
0


In [25]:
# Quick example of **kwargs

def print_details(**kwargs):
    for key, value in kwargs.items():
        print(f"{key}: {value}")

# Calling the function with different keyword arguments
print_details(name="Alice", age=30, city="Barcelona")
print_details(product="Laptop", price=1200)
print(print_details())

name: Alice
age: 30
city: Barcelona
product: Laptop
price: 1200
None


In [26]:
def decorator_function_print_return_type(func):
  # Define wrapper(), the decorated function
  # This function will print the type of the return value of the function being decorated
  # Accepts any number of positional (*arg) and keyword arguments (*kwargs) that the function 'func' being decorated can accept
  def wrapper(*args, **kwargs): 
    # Call the function being decorated
    result = func(*args, ** kwargs)
    print('{}() returned type {}'.format(func.__name__, type(result))) # __name__ is the name of the function being decorated
    return result
  # Return the decorated function (wrapper) to be used in place of the original function 'func'
  return wrapper
  
@decorator_function_print_return_type # This is the same as saying simple_function = decorator_function_print_return_type(simple_function)
def simple_function(*args, **kwargs):
  print("Positional arguments:", args)
  print("Keyword arguments:", kwargs)
  return args, kwargs  # Returning args and kwargs for illustration
  
print(simple_function(42))        # Output: (42,) {}
print(simple_function([1, 2, 3])) # Output: ([1, 2, 3],)
print(simple_function({'a': 42})) # Output: ({'a': 42},)

print(simple_function(42, name="Alice", age=30))                     # Output: (42,) {'name': 'Alice', 'age': 30}
print(simple_function([1, 2, 3], city="Barcelona", country="Spain")) # Output: ([1, 2, 3],) {'city': 'Barcelona', 'country': 'Spain'}
print(simple_function({'a': 42}, 100))                               # Output: ({'a': 42},) {100}

Positional arguments: (42,)
Keyword arguments: {}
simple_function() returned type <class 'tuple'>
((42,), {})
Positional arguments: ([1, 2, 3],)
Keyword arguments: {}
simple_function() returned type <class 'tuple'>
(([1, 2, 3],), {})
Positional arguments: ({'a': 42},)
Keyword arguments: {}
simple_function() returned type <class 'tuple'>
(({'a': 42},), {})
Positional arguments: (42,)
Keyword arguments: {'name': 'Alice', 'age': 30}
simple_function() returned type <class 'tuple'>
((42,), {'name': 'Alice', 'age': 30})
Positional arguments: ([1, 2, 3],)
Keyword arguments: {'city': 'Barcelona', 'country': 'Spain'}
simple_function() returned type <class 'tuple'>
(([1, 2, 3],), {'city': 'Barcelona', 'country': 'Spain'})
Positional arguments: ({'a': 42}, 100)
Keyword arguments: {}
simple_function() returned type <class 'tuple'>
(({'a': 42}, 100), {})


In [27]:
# Creating an attribute for the function that counts the number of times it was called

def count_calls(func):
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        print(f"Call {wrapper.count} of {func.__name__}")
        return func(*args, **kwargs)
    wrapper.count = 0
    return wrapper

@count_calls
def say_hello():
    print("Hello!")

say_hello()
say_hello()
say_hello()

print('say_hello() was called {} times.'.format(say_hello.count))



Call 1 of say_hello
Hello!
Call 2 of say_hello
Hello!
Call 3 of say_hello
Hello!
say_hello() was called 3 times.


### Preserving the Metadata of the Function you are Decorating

In [28]:
# Example of a Decorator function that DOES NOT preserve the docstring of the original function

def decorator_function(func):
  def wrapper(*args, **kwargs):
    """This is the docstring of the wrapper function.""" # This docstring will replace the original function's docstring
    result = func(*args, **kwargs)  # Call the original function
    print("This is the decorated function.")
    return result
  return wrapper

@decorator_function
def original_function():
    """This is a docstring from the original function."""
    return 42

print(original_function.__doc__)  # Output: This is the docstring of the wrapper function.

This is the docstring of the wrapper function.


In [29]:
# Decorator function that preserves the docstring of the original function

from functools import wraps

def decorator_function(func):
  # Decorate wrapper() so that it keeps func()'s metadata
  @wraps(func) # ensures that wrapper keeps the original function’s metadata (e.g., __name__, __doc__)
  def wrapper(*args, **kwargs):
    """This is the docstring of the wrapper function.""" # This is a new docstring that will be overwritten by the @wraps decorator
    result = func(*args, **kwargs)  # Call the original function
    print("This is the decorated function.")
    return result
  return wrapper

@decorator_function
def original_function():
    """This is a docstring from the original function."""
    return 42

print(original_function.__doc__)  # Output: This is the docstring of the wrapper function.

This is a docstring from the original function.


In [30]:
# Decorator function that preserves result of the original function

from functools import wraps

def add_hello(func):
  # Decorate wrapper() so that it keeps func()'s metadata
  @wraps(func) # ensures that wrapper keeps the original function’s metadata (e.g., __name__, __doc__)
  def wrapper(*args, **kwargs):
    print('Hello')
    return func(*args, **kwargs)
  return wrapper
  
@add_hello
def print_sum(a, b):
  """This is a docstring from the original function."""
  print(a + b)
  
# print_sum(10, 20)   # Output: If you do not use @wraps, it will print 'Hello' and then the sum of the numbers.
print_sum(10, 20)   # Output: If you use @wraps, it will print only the sum of the numbers.

print(print_sum.__doc__)  # Output: This is a docstring from the original function.

Hello
30
This is a docstring from the original function.


* Note:
    * `__wrapped__`
        * you need to use `functools.wraps` to explicitly set it
        * The `__wrapped__` attribute is used when a function has been wrapped by a decorator. 
        * **It gives you access to the original, undecorated function!**
        * When you decorate a function, Python replaces it with a new function (the wrapper). But sometimes, you may want to access the original function for debugging, testing, or introspection.

In [31]:
# Accessing the original function from a decorated function

from functools import wraps

def add_hello(func):
  # Decorate wrapper() so that it keeps func()'s metadata
  @wraps(func) # ensures that wrapper keeps the original function’s metadata (e.g., __name__, __doc__)
  def wrapper(*args, **kwargs):
    print('Hello')
    return func(*args, **kwargs)
  return wrapper
  
@add_hello
def print_sum(a, b):
  """This is a docstring from the original function."""
  print("This is the sum of a and b:")
  return(a + b)
  
# print_sum(10, 20)   # Output: If you do not use @wraps, it will print 'Hello' and then the sum of the numbers.
print_sum(10, 20)   # Output: If you use @wraps, it will print only the sum of the numbers.

print(print_sum.__doc__)  # Output: This is a docstring from the original function.

print(print_sum.__name__)       # Outputs: print_sum (not wrapper)
print(print_sum.__wrapped__)    # Outputs: <function print_sum at 0x...>
print(print_sum.__wrapped__(10, 20))  # Outputs: 30



Hello
This is the sum of a and b:
This is a docstring from the original function.
print_sum
<function print_sum at 0x000001DB3213A3E0>
This is the sum of a and b:
30


### Using Decorators to check the time it takes to run generic functions

In [32]:
import time
from functools import wraps

def timing_decorator(func):
    @wraps(func)  # ensures that wrapper keeps the original function’s metadata (e.g., __name__, __doc__)
    def wrapper(*args, **kwargs):
        start_time = time.time()  # Start timer
        result = func(*args, **kwargs)  # Call the original function
        end_time = time.time()  # End timer
        print(f"Function '{func.__name__}' took {end_time - start_time:.5f} seconds to run")
        return result  # Ensure function's original return value is preserved
    return wrapper

@timing_decorator
def slow_function():
    time.sleep(2)  # Simulating a slow process
    print("Finished slow function")

@timing_decorator
def add_numbers(a, b):
    time.sleep(1)  # Simulating delay
    return a + b

# Run decorated functions
slow_function()
print(add_numbers(5, 10))

Finished slow function
Function 'slow_function' took 2.00069 seconds to run
Function 'add_numbers' took 1.00114 seconds to run
15


### Creating Decorators (Generalizing a Decorator to take arguments)

In [33]:
# Decorator function that takes an argument

def run_n_times(n):
    def decorator_function(func):
        def wrapper(*args, **kwargs):
            for _ in range(n):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator_function

@run_n_times(3)
def print_sum(a, b):
    print(a + b)

print_sum(3, 5) # Outputs: 8, 8, 8

8
8
8


In [34]:
# Now, let's create a decorator that takes an argument

def run_n_times(n):
    def decorator_function(func):
        def wrapper(*args, **kwargs):
            for _ in range(n):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator_function

run_3_times = run_n_times(3)

@run_3_times
def print_sum(a, b):
    print(a + b)

print_sum(3, 5) # Outputs: 8, 8, 8

8
8
8


### Using Decorators to Tag Functions

* This code creates a custom decorator called @tag that adds metadata (tags) to a function.
* When applied, it allows functions to store custom tags that can be accessed later.

In [35]:
def tag(*tags): # This is not the actual decorator yet, it is a function that returns a decorator
  def decorator(func): # This is the real decorator that wraps around the function we decorate
    # Ensure the decorated function keeps its metadata
    @wraps(func) # ensures that wrapper keeps the original function’s metadata (e.g., __name__, __doc__)
    def wrapper(*args, **kwargs): # The wrapper function calls the original function (func) and returns its result
      return func(*args, **kwargs)
    wrapper.tags = tags # here, we add a custom attribute (tags) to the wrapped function (tags stores the arguments passed when calling @tag(...))
    return wrapper      # Return the decorated function
  return decorator

@tag('test', 'this is a tag') # The decorator function is applied to foo(), making it the wrapped function
def simple_function():
  pass

print(simple_function.tags)

('test', 'this is a tag')


### Using Decorators to Check for Data Types returned by Functions

* This code defines a decorator (@returns_dict) that ensures a function always returns a dictionary (dict). 
* If the function does not return a dictionary, an assertion error is raised
* Useful for ensuring consistent return types in APIs, data pipelines, and more.

In [36]:
# Checking if a function returns a dictionary

def returns_dict(func):  # This is the decorator
  def wrapper(value):    # The wrapper function calls the original function (func) and returns its result
    result = func(value)
    assert type(result) == dict # Ensure the return type is a dictionary
    return result               # Return the result if it's a dict
  return wrapper  # Return the decorated function
  
@returns_dict  # this decorator modifies simple_function, enforcing that it must return a dictionary
def simple_function(value):
  return value

try:
  print(simple_function([1,2,3]))
except AssertionError:
  print('simple_function() did not return a dict!')
  

simple_function() did not return a dict!


In [38]:
# Expading the Decorator to take a type and check if the function returns this type

def returns(return_type): # This is not the actual decorator yet, it is a function that returns a decorator
  def decorator(func):    # This is the real decorator that wraps around the function we decorate
    def wrapper(value):   # The wrapper function calls the original function (func) and returns its result
      result = func(value)
      assert type(result) == return_type # Ensure the return type is a dictionary
      return result                      # Return the result if it's a dict
    return wrapper  # Return the decorated function
  return decorator
  
@returns(list)  # this decorator modifies simple_function, enforcing that it must return a dictionary
def simple_function(value):
  return value

try:
  print(simple_function([1,2,3]))
except AssertionError:
  print('simple_function() did not return a dict!')
  

[1, 2, 3]
